In [10]:
import pandas as pd
import seaborn as sns
import plotly.express as px


# Load in and preprocess Datasets (Fantasy, Combine, Advanced Statistics)

### Fantasy Data (trailing 10 years)

In [81]:
# Import Fantasy data from Pro Football Reference

years_for_pfr_data = ['2021',
                    '2020',
                    '2019',
                    '2018',
                    '2017',
                    '2016',
                    '2015',
                    '2014',
                    '2013',
                    '2012',
                    ]

fantasy_dfs = []
for year in years_for_pfr_data: 
    link = "https://www.pro-football-reference.com/years/" + year + "/fantasy.htm" 
    df = pd.read_html(link)[0]
    df['year'] = year
    fantasy_dfs.append(df)
    
fantasy_df = pd.concat(fantasy_dfs).reset_index(drop=True)

# Flatten dataframe
fantasy_df.columns = fantasy_df.columns.get_level_values(0) + '_' +  fantasy_df.columns.get_level_values(1)

## Preprocess data
# Rename Columns
new_col_names = {'Unnamed: 0_level_0_Rk': 'Rank',
        'Unnamed: 1_level_0_Player': 'Player',
        'Unnamed: 2_level_0_Tm': 'Team',
        'Unnamed: 3_level_0_FantPos': 'Position',
        'Unnamed: 4_level_0_Age': 'Age',
        'year_': 'Year'}
        
fantasy_df.rename(columns=new_col_names, inplace=True)

# Drop rows where column names are printed again (formatting issue w/ PFR website)
fantasy_df = fantasy_df[~fantasy_df['Rank'].isin(['Rk'])]

# Drop rows where Position is null (formatting issue w/ PFR website)
fantasy_df = fantasy_df[fantasy_df['Position'].notna()]

# Clean Player Name Column
fantasy_df['Player'] = fantasy_df['Player'].str.strip('*+')

# Fill NaN cells with 0
fantasy_df = fantasy_df.fillna(0)

# Rank Overall Fantasy Rank by Year (All values weren't filled in initally, must be added to dataset in place of zeros)
# ...

# Update Datatypes for columns
convert_dtype_dict = {'Rank': int,
                'Player': str,
                'Team': str,
                'Position': str,
                'Age': int,
                'Games_G': int,
                'Games_GS': int,
                'Passing_Cmp': int,
                'Passing_Att': int,
                'Passing_Yds': int,
                'Passing_TD': int,
                'Passing_Int': int,
                'Rushing_Att': int,
                'Rushing_Yds': int,
                'Rushing_Y/A': float,
                'Rushing_TD': int,
                'Receiving_Tgt': int,
                'Receiving_Rec': int,
                'Receiving_Yds': int,
                'Receiving_Y/R': float,
                'Receiving_TD': int,
                'Fumbles_Fmb': int,
                'Fumbles_FL': int,
                'Scoring_TD': int,
                'Scoring_2PM': int,
                'Scoring_2PP': int,
                'Fantasy_FantPt': int,
                'Fantasy_PPR': float,
                'Fantasy_DKPt': float,
                'Fantasy_FDPt': float,
                'Fantasy_VBD': float,
                'Fantasy_PosRank': int,
                'Fantasy_OvRank': int,
                'Year': str
                }
 
fantasy_df = fantasy_df.astype(convert_dtype_dict)
# Uncomment to Save dataset as flatfile (if desired)
fantasy_df.to_csv('data/pfr_10yr_data_2022.csv')

### Combine Data

In [134]:
# Import Combine Data from Pro Football Reference

years_for_pfr_data = ['2021',
                    '2020',
                    '2019',
                    '2018',
                    '2017',
                    '2016',
                    '2015',
                    '2014',
                    '2013',
                    '2012',
                    '2011',
                    '2010',
                    '2009',
                    '2008',
                    '2007',
                    '2006',
                    '2005',
                    '2004',
                    '2003',
                    '2002',
                    '2001',
                    '2000',
                    ]

combine_dfs = []
for year in years_for_pfr_data: 
    link = "https://www.pro-football-reference.com/draft/" + year + "-combine.htm" 
    df = pd.read_html(link)[0]
    df['Draft_Year'] = year
    combine_dfs.append(df)
    
combine_df = pd.concat(combine_dfs).reset_index(drop=True)

## Preprocess Data

# Drop rows where column names are printed again (formatting issue w/ PFR website)
combine_df = combine_df[~combine_df['Player'].isin(['Player'])]

# Drop Unnecessary columns
combine_df = combine_df.drop(['Pos','College','Drafted (tm/rnd/yr)'], axis=1)

# Fill NaN cells with 0
combine_df = combine_df.fillna(0)

# Update Datatypes for columns
convert_dtype_dict = {'Player': str,
                'School': str,
                'Ht': str,
                'Wt': int,
                '40yd': float,
                'Vertical': float,
                'Bench': int,
                'Broad Jump': int,
                '3Cone': float,
                'Shuttle': float,
                'Draft_Year': str,
                }
combine_df = combine_df.astype(convert_dtype_dict)

combine_df.rename(columns={'Broad Jump': 'BroadJump'}, inplace=True)

# Uncomment to Save combine dataset as flatfile (if desired)
combine_df.to_csv('data/pfr_combine_data_2000_thru_2022.csv')


# Advanced Analytic Data


## Join in other data specific to players (combine results, advanced analytics, etc)

In [135]:
final_df = fantasy_df.merge(combine_df, how='left', on='Player')

In [30]:
# Number 1 ranked players from each fantasy season
fantasy_df[fantasy_df['Fantasy_OvRank']==1]

,Rank,Player,Team,Position,Age,Games_G,Games_GS,Passing_Cmp,Passing_Att,Passing_Yds,...,Scoring_2PM,Scoring_2PP,Fantasy_FantPt,Fantasy_PPR,Fantasy_DKPt,Fantasy_FDPt,Fantasy_VBD,Fantasy_PosRank,Fantasy_OvRank,Year
0,1,Jonathan Taylor,IND,RB,22,17,17,0,0,0,...,0,0,333,373.1,381.1,353.1,187.0,1,1,2021
693,1,Derrick Henry,TEN,RB,26,16,16,0,0,0,...,1,0,314,333.1,341.1,323.6,184.0,1,1,2020
1372,1,Christian McCaffrey,CAR,RB,23,16,16,0,2,0,...,1,0,355,471.2,477.2,413.2,215.0,1,1,2019
2011,1,Todd Gurley,LAR,RB,24,14,14,0,0,0,...,3,0,313,372.1,379.1,342.6,178.0,1,1,2018
2648,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,0,0,319,383.3,391.3,351.3,192.0,1,1,2017
3265,1,David Johnson,ARI,RB,25,16,16,0,0,0,...,1,0,328,407.8,416.8,367.8,191.0,1,1,2016
3869,1,Devonta Freeman,ATL,RB,23,15,13,0,0,0,...,0,0,243,316.4,324.4,279.9,123.0,1,1,2015
4477,1,DeMarco Murray,DAL,RB,26,16,16,0,0,0,...,0,0,294,351.1,362.1,322.6,173.0,1,1,2014
5082,1,Jamaal Charles,KAN,RB,27,15,15,0,0,0,...,0,0,308,378.0,386.0,343.0,182.0,1,1,2013
5686,1,Adrian Peterson,MIN,RB,27,16,16,0,0,0,...,1,0,307,347.4,355.4,327.4,191.0,1,1,2012


In [31]:
# One players Fantasy Prodution during career
fantasy_df[fantasy_df['Player']=="Todd Gurley"]

,Rank,Player,Team,Position,Age,Games_G,Games_GS,Passing_Cmp,Passing_Att,Passing_Yds,...,Scoring_2PM,Scoring_2PP,Fantasy_FantPt,Fantasy_PPR,Fantasy_DKPt,Fantasy_FDPt,Fantasy_VBD,Fantasy_PosRank,Fantasy_OvRank,Year
762,68,Todd Gurley,ATL,RB,26,15,15,0,0,0,...,0,0,138,163.2,169.2,150.7,8.0,23,68,2020
1395,24,Todd Gurley,LAR,RB,25,15,15,0,0,0,...,1,0,188,219.4,227.4,203.9,48.0,12,24,2019
2011,1,Todd Gurley,LAR,RB,24,14,14,0,0,0,...,3,0,313,372.1,379.1,342.6,178.0,1,1,2018
2648,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,0,0,319,383.3,391.3,351.3,192.0,1,1,2017
3315,50,Todd Gurley,LAR,RB,22,16,16,0,0,0,...,0,0,155,198.2,205.2,176.7,18.0,20,50,2016
3882,14,Todd Gurley,STL,RB,21,13,12,0,0,0,...,0,0,187,208.4,215.4,197.9,67.0,5,14,2015


In [9]:
# Functions located in this cell
#...

# Plots

In [113]:
## Line plot of Age vs Fantasy PPR points (line by position)
# Groupby on Data
top_20_pos_rk_players = fantasy_df[fantasy_df['Fantasy_PosRank'] <= 20]
age_pos_avg_ppr_gb = top_20_pos_rk_players.groupby(['Age','Position']).mean('Fantasy_PPR').reset_index()

# Seaborn

#age_pos_ppr_fig = sns.lineplot(data=age_pos_avg_ppr_gb, x='Age', y='Fantasy_PPR', hue='Position')
#age_pos_ppr_fig.set_title('Average Fantasy Points (PPR) by Age and Position')
#age_pos_ppr_fig.set_xlabel('Age')
#age_pos_ppr_fig.set_ylabel('Fantasy Points')

# Plotly
age_pos_ppr_fig_px = px.line(age_pos_avg_ppr_gb, x="Age", y="Fantasy_PPR", color='Position', title='Average Fantasy Points (PPR) by Age and Position')
plot_filename='plots/avg_points_by_age_position.html'
age_pos_ppr_fig_px.write_html(plot_filename)
age_pos_ppr_fig_px.show()

In [114]:
## Line plot of count on top 20 players at each position by Age (line by position)
# Groupby on Data
top_20_pos_rk_players = fantasy_df[fantasy_df['Fantasy_PosRank'] <= 20]
age_pos_count_gb = top_20_pos_rk_players.groupby(['Age','Position']).count().reset_index()

# Plotly
posrnk_age_count_plot_px = px.line(age_pos_count_gb, x="Age", y="Rank", color='Position', title='Number of Top 20 Position Rankings by Age and Position')
posrnk_age_count_plot_px.update_layout(
                   xaxis_title='Age',
                   yaxis_title='Count of Top 20 Players')
plot_filename='plots/top20_player_count_by_age_position.html'
posrnk_age_count_plot_px.write_html(plot_filename)
posrnk_age_count_plot_px.show()

In [115]:
## Scatter plot of Position Rank vs Fantasy PPR points (Color by Position)

# Seaborn
#posrnk_ppr_plot_sns = sns.scatterplot(data=final_df, x='Fantasy_PosRank', y='Fantasy_PPR', hue='Position')
#posrnk_ppr_plot_sns.set_title('Average Fantasy Points (PPR) by Position')
#posrnk_ppr_plot_sns.set_xlabel('Position Rank')
#posrnk_ppr_plot_sns.set_ylabel('Fantasy Points')

# Plotly
posrnk_ppr_plot_px = px.scatter(fantasy_df, x="Fantasy_PosRank", y="Fantasy_PPR", color='Position', hover_data=['Player', 'Year'], title='Fantasy Points (PPR) by Position and Rank')
plot_filename='plots/points_by_position.html'
posrnk_ppr_plot_px.write_html(plot_filename)
posrnk_ppr_plot_px.show()

In [116]:
# Boxplot of position vs Fantasy points
# Seaborn
#box_pos_ppr_sns = sns.boxplot(data=final_df, x='Position', y='Fantasy_PPR')

# Plotly
box_pos_ppr_px = px.box(fantasy_df, x="Position", y="Fantasy_PPR", hover_data=['Player', 'Year'], title='Fantasy Points by Position')
plot_filename='plots/boxplot_points_by_position.html'
box_pos_ppr_px.write_html(plot_filename)
box_pos_ppr_px.show()

In [117]:
#rbs_speed = final_df[(final_df['Position'] == 'RB') & (final_df['40yd'] > 0)]
#rb_speed_ppr_px = px.scatter(rbs_speed, x='40yd', y='Fantasy_PPR', hover_data=['Player'])
#rb_speed_ppr_px.show()

In [132]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
qbs_speed = final_df[(final_df['Position'] == 'QB') & (final_df['40yd'] > 0)]
rbs_speed = final_df[(final_df['Position'] == 'RB') & (final_df['40yd'] > 0)]
wrs_speed = final_df[(final_df['Position'] == 'WR') & (final_df['40yd'] > 0)]
tes_speed = final_df[(final_df['Position'] == 'TE') & (final_df['40yd'] > 0)]

speed_and_pos_vs_points = make_subplots(rows=2, cols=2, subplot_titles=('Quarterback', 'Running Back', 'Wide Receiver', 'Tight End'))

speed_and_pos_vs_points.add_trace(
    go.Scatter(
        x=qbs_speed['40yd'],
        y=qbs_speed['Fantasy_PPR'],
        mode='markers'
    ),
    row=1, col=1
)

speed_and_pos_vs_points.add_trace(
    go.Scatter(
        x=rbs_speed['40yd'],
        y=rbs_speed['Fantasy_PPR'],
        mode='markers'
    ),
    row=1, col=2
)

speed_and_pos_vs_points.add_trace(
    go.Scatter(
        x=wrs_speed['40yd'],
        y=wrs_speed['Fantasy_PPR'],
        mode='markers'
    ),
    row=2, col=1
)

speed_and_pos_vs_points.add_trace(
    go.Scatter(
        x=tes_speed['40yd'],
        y=tes_speed['Fantasy_PPR'],
        mode='markers'
    ),
    row=2, col=2
)

# Update xaxis properties
speed_and_pos_vs_points.update_xaxes(title_text="40-yd Dash time (secs)", range=[4, 6], row=1, col=1)
speed_and_pos_vs_points.update_xaxes(title_text="40-yd Dash time (secs)", range=[4, 6], row=1, col=2)
speed_and_pos_vs_points.update_xaxes(title_text="40-yd Dash time (secs)", range=[4, 6], row=2, col=1)
speed_and_pos_vs_points.update_xaxes(title_text="40-yd Dash time (secs)", range=[4, 6], row=2, col=2)

# Update yaxis properties
speed_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=1, col=1)
speed_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=1, col=2)
speed_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=2, col=1)
speed_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=2, col=2)

speed_and_pos_vs_points.update_layout(title_text="Fantasy Points (PPR) vs 40-Time by Position", height=600, showlegend=False)

plot_filename='plots/speed_vs_points_and_position.html'
speed_and_pos_vs_points.write_html(plot_filename)
speed_and_pos_vs_points.show()

In [133]:
qbs_vert = final_df[(final_df['Position'] == 'QB') & (final_df['Vertical'] > 0)]
rbs_vert = final_df[(final_df['Position'] == 'RB') & (final_df['Vertical'] > 0)]
wrs_vert = final_df[(final_df['Position'] == 'WR') & (final_df['Vertical'] > 0)]
tes_vert = final_df[(final_df['Position'] == 'TE') & (final_df['Vertical'] > 0)]

vert_and_pos_vs_points = make_subplots(rows=2, cols=2, subplot_titles=('Quarterback', 'Running Back', 'Wide Receiver', 'Tight End'))

vert_and_pos_vs_points.add_trace(
    go.Scatter(
        x=qbs_vert['Vertical'],
        y=qbs_vert['Fantasy_PPR'],
        mode='markers'
    ),
    row=1, col=1
)

vert_and_pos_vs_points.add_trace(
    go.Scatter(
        x=rbs_vert['Vertical'],
        y=rbs_vert['Fantasy_PPR'],
        mode='markers'
    ),
    row=1, col=2
)

vert_and_pos_vs_points.add_trace(
    go.Scatter(
        x=wrs_vert['Vertical'],
        y=wrs_vert['Fantasy_PPR'],
        mode='markers'
    ),
    row=2, col=1
)

vert_and_pos_vs_points.add_trace(
    go.Scatter(
        x=tes_vert['Vertical'],
        y=tes_vert['Fantasy_PPR'],
        mode='markers'
    ),
    row=2, col=2
)

# Update xaxis properties
vert_and_pos_vs_points.update_xaxes(title_text="Vertical Jump (inches)", range=[10, 50], row=1, col=1)
vert_and_pos_vs_points.update_xaxes(title_text="Vertical Jump (inches)", range=[10, 50], row=1, col=2)
vert_and_pos_vs_points.update_xaxes(title_text="Vertical Jump (inches)", range=[10, 50], row=2, col=1)
vert_and_pos_vs_points.update_xaxes(title_text="Vertical Jump (inches)", range=[10, 50], row=2, col=2)

# Update yaxis properties
vert_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=1, col=1)
vert_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=1, col=2)
vert_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=2, col=1)
vert_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=[0, 500], row=2, col=2)

vert_and_pos_vs_points.update_layout(title_text="Fantasy Points (PPR) vs Vertical Jump by Position", height=600, showlegend=False)

plot_filename='plots/vert_vs_points_and_position.html'
vert_and_pos_vs_points.write_html(plot_filename)
vert_and_pos_vs_points.show()

In [147]:
def pts_vs_combine_scatterplots(df, subplot_config, yaxis_range=[0,500]):
    for plot in subplot_config['plots']:
        print("Using "+plot['args']['measurement']+" for plot...")
        measure = plot['args']['measurement']
        unit = plot['args']['units']
        xrange = plot['args']['xrange']

        qbs = df[(final_df['Position'] == 'QB') & (df[measure] > 0)]
        rbs = df[(final_df['Position'] == 'RB') & (df[measure] > 0)]
        wrs = df[(final_df['Position'] == 'WR') & (df[measure] > 0)]
        tes = df[(final_df['Position'] == 'TE') & (df[measure] > 0)]

        meas_and_pos_vs_points = make_subplots(rows=2, cols=2, subplot_titles=('Quarterback', 'Running Back', 'Wide Receiver', 'Tight End'))

        meas_and_pos_vs_points.add_trace(
            go.Scatter(
                x=qbs[measure],
                y=qbs['Fantasy_PPR'],
                mode='markers'
            ),
            row=1, col=1
        )

        meas_and_pos_vs_points.add_trace(
            go.Scatter(
                x=rbs['Vertical'],
                y=rbs['Fantasy_PPR'],
                mode='markers'
            ),
            row=1, col=2
        )

        meas_and_pos_vs_points.add_trace(
            go.Scatter(
                x=wrs['Vertical'],
                y=wrs['Fantasy_PPR'],
                mode='markers'
            ),
            row=2, col=1
        )

        meas_and_pos_vs_points.add_trace(
            go.Scatter(
                x=tes['Vertical'],
                y=tes['Fantasy_PPR'],
                mode='markers'
            ),
            row=2, col=2
        )

        # Update xaxis properties
        x_title = str(measure)+' ('+str(unit)+')'
        meas_and_pos_vs_points.update_xaxes(title_text=x_title, range=xrange, row=1, col=1)
        meas_and_pos_vs_points.update_xaxes(title_text=x_title, range=xrange, row=1, col=2)
        meas_and_pos_vs_points.update_xaxes(title_text=x_title, range=xrange, row=2, col=1)
        meas_and_pos_vs_points.update_xaxes(title_text=x_title, range=xrange, row=2, col=2)

        # Update yaxis properties
        meas_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=yaxis_range, row=1, col=1)
        meas_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=yaxis_range, row=1, col=2)
        meas_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=yaxis_range, row=2, col=1)
        meas_and_pos_vs_points.update_yaxes(title_text="Fantasy Points (PPR)", range=yaxis_range, row=2, col=2)

        meas_and_pos_vs_points.update_layout(title_text="Fantasy Points (PPR) vs "+str(measure)+" by Position", height=600, showlegend=False)

        plot_filename='plots/'+str(measure)+'_vs_points_and_position.html'
        meas_and_pos_vs_points.write_html(plot_filename)
        print("Plot saved")
    print('')
    print('Plots complete!')


subplot_config = \
{
    "plots":[
        {
            "args": {
                "measurement":"40yd",
                "units": 'secs',
                "xrange": [4,6]
            }
        },
        {
            "args": {
                "measurement": "Vertical",
                "units": 'inches',
                "xrange": [20,50]
            }
        },
        {
            "args": {
                "measurement": "Bench",
                "units": 'reps',
                "xrange":[0,50]
            }
        },
        {
            "args": {
                "measurement": "BroadJump",
                "units": 'inches',
                "xrange": [80,150]
            }
        },
        {
            "args": {
                "measurement": "3Cone",
                "units": 'secs',
                "xrange": [6,9]
            }
        },
        {
            "args": {
                "measurement": "Shuttle",
                "units": 'secs',
                "xrange": [3.5,5.5]
            }
        }
    ]
}


pts_vs_combine_scatterplots(final_df, subplot_config)

Using 40yd for plot...
Plot saved
Using Vertical for plot...
Plot saved
Using Bench for plot...
Plot saved
Using BroadJump for plot...
Plot saved
Using 3Cone for plot...
Plot saved
Using Shuttle for plot...
Plot saved

Plots complete!
